# Overview

This notebook imports raw ws3 input data, reformats and monkey-patches the data, and exports Woodstock formatted input data files (which we will use in other DSS notebooks for this case as the input data files). 

# Set up environment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import ws3.forest, ws3.core
import csv
import numpy as np
# from util import schedule_harvest_areacontrol, schedule_harvest_areacontrol_asap, schedule_harvest_areacontrol_null

Define some key model parameters (will get used but defined here up top for convenience).

In [3]:
period_length = 10
max_age =  1000

# Import and reformat inventory and yield input data

Read forest inventory data into memory (vector polygon GIS data layer with attribute table, in ESRI Shapefile format). This dataset represents a small subset of timber supply area (TSA) 17 in British Columbia. We monkey-patch the inventory data here to make it line up nicely with what we need downstream as input for the ws3 model (i.e., changes we make here to the in-memory dataset are not saved to the original dataset on disk). Most of what we are doing here is setting up the _theme_ columns in the attribute table, which should help newer ws3 users make the connection between input data and the landscape themes in ws3 model further down.

In [14]:
stands = gpd.read_file('data/tsa04contclass/tsa04.shp')
# stands = stands.rename(columns={'thlb':'theme1', 'au':'theme2', 'ldspp':'theme3', 'age2015':'age', 'shape_area':'area' })
# stands['area'] = stands.geometry.area * 0.0001 # monkey-patch broken area attribute
# stands.insert(4, 'theme4', stands['theme2'])
# stands['theme2'] = stands['theme2'].astype(int)
stands

,FID_Golden,FID_Gold_1,ZONE,SUBZONE,VARIANT,PHASE,NTRLDSTRBN,MAP_LABEL,FID_Gold_2,HERD_STATU,...,ORIG_FID,Shape_Leng,Shape_Area,contclass,rollup,netdown,THLB_Area,Age_2023,AU,geometry
0,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,468,892.970730,16474.221678,C,THLB,THLB,1.647422,46,6,"POLYGON ((651330.000 1473337.500, 651330.000 1..."
1,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,468,847.015427,25214.104031,C,THLB,THLB,2.521410,46,6,"POLYGON ((651251.113 1473060.000, 651250.257 1..."
2,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,468,2146.240646,87035.420516,C,THLB,THLB,8.703542,46,6,"POLYGON ((651090.000 1473000.000, 651090.000 1..."
3,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,468,255.813594,2305.734220,C,THLB,THLB,0.230573,46,6,"POLYGON ((650746.974 1472340.000, 650640.000 1..."
4,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,469,859.337139,15510.057930,C,THLB,THLB,1.551006,46,6,"POLYGON ((651390.000 1473330.000, 651390.000 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63203,1,7,BWBS,dk,None,None,NDT3,BWBSdk,-1,None,...,71281,326.907495,1502.856584,N,6_Riparian,6_01_Stream_Buffer,0.000000,83,33,"POLYGON ((649003.440 1506715.961, 649006.877 1..."
63204,1,7,BWBS,dk,None,None,NDT3,BWBSdk,-1,None,...,71281,476.065471,1950.527358,N,6_Riparian,6_01_Stream_Buffer,0.000000,83,33,"POLYGON ((648270.000 1506662.850, 648270.000 1..."
63205,1,7,BWBS,dk,None,None,NDT3,BWBSdk,-1,None,...,71281,415.325791,1606.933608,N,6_Riparian,6_01_Stream_Buffer,0.000000,83,33,"POLYGON ((649050.000 1506509.390, 649050.000 1..."
63206,1,11,MH,mm,2,None,NDT1,MHmm2,-1,None,...,71292,287.561926,1279.447945,N,6_Riparian,6_01_Stream_Buffer,0.000000,101,33,"POLYGON ((622490.804 1495979.904, 622468.035 1..."


In [49]:
columns_to_keep = ['TSA_NUMBER', 'contclass', 'Age_2023', 'AU', 'geometry']
stands_modified = stands[columns_to_keep].copy()
stands_modified.loc[:,'area'] = stands_modified.geometry.area * 0.0001 # monkey-patch broken area attribute
stands_modified =  stands_modified.rename(columns={'TSA_NUMBER': 'theme0', 'contclass':'theme1', 'AU':'theme2', 'Age_2023':'age'})
stands_modified = stands_modified.drop(columns='geometry')
stands_modified.insert(4, 'theme3', stands_modified['theme2'])
stands_modified.insert(5, 'theme4', stands_modified['theme2']) # to be filled out with the scpecies code
stands_modified.insert(5, 'age', stands_modified.pop('age'))
stands_modified

,theme0,theme1,theme2,theme3,theme4,age,area
0,04,C,6,6,6,46,1.647422
1,04,C,6,6,6,46,2.521410
2,04,C,6,6,6,46,8.703542
3,04,C,6,6,6,46,0.230573
4,04,C,6,6,6,46,1.551006
...,...,...,...,...,...,...,...
63203,04,N,33,33,33,83,0.150252
63204,04,N,33,33,33,83,0.195038
63205,04,N,33,33,33,83,0.160693
63206,04,N,33,33,33,101,0.127911


Read yield data from a CSV file and recast AU column data type to integer.

In [68]:
stands_modified['area'].sum()

191273.58586001678

In [56]:
df = pd.read_feather('vdyp_curves_smooth-tsa04.feather')
df

,index,age,volume,stratum_code,si_level
0,17,18,6.282000e-11,ESSF_BL,L
1,18,19,3.625496e-08,ESSF_BL,L
2,19,20,7.916861e-07,ESSF_BL,L
3,20,21,6.105747e-06,ESSF_BL,L
4,21,22,2.801684e-05,ESSF_BL,L
...,...,...,...,...,...
12296,294,295,9.945361e+01,BWBS_AT,H
12297,295,296,9.878314e+01,BWBS_AT,H
12298,296,297,9.811513e+01,BWBS_AT,H
12299,297,298,9.744960e+01,BWBS_AT,H


In [50]:
yld = pd.read_csv('data/yld.csv')
yld['AU'] = yld['AU'].astype(int)

Create analysis unit (AU) dataframe from stands dataframe data.

In [65]:
AU = pd.DataFrame(stands_modified['theme2']).drop_duplicates()
AU.rename(columns={'theme2':'AU'}, inplace=True)
len(AU), AU

(28,
        AU
 0       6
 10     22
 12     13
 30     29
 56     33
 96     19
 146    10
 388    26
 412     3
 498    25
 802     7
 922    31
 1154   18
 1700   24
 1912   23
 2268    1
 2890   30
 3828   12
 4244    5
 4504   16
 5026   21
 7412    4
 12893   2
 14077  17
 27490  28
 28596  14
 28928  15
 34010  32)

Join `AU` and `yld` dataframes.

In [67]:
yldmerged = pd.merge(AU, yld, on=['AU'], how='inner')
yldmerged

,AU,Yieldscode,THLB,LDSPP,SPCode,Wdks,X0,X10,X20,X30,...,X260,X270,X280,X290,X300,X310,X320,X330,X340,X350
0,22,*Y,?,Poplar,POVOL,1,0.0,0.0,0.0,1.2,...,186.7,187.7,188.7,189.6,190.5,191.1,191.2,191.2,191.2,191.2
1,29,*Y,?,Paper birch,PBVOL,1,0.0,0.0,0.0,0.0,...,180.1,180.2,180.7,181.2,181.7,181.9,181.9,181.9,181.9,181.9
2,26,*Y,?,Cottonwood,BCVOL,1,0.0,0.0,0.0,0.2,...,357.4,359.2,361.0,362.6,364.1,365.6,365.9,365.9,365.9,365.9
3,3,*Y,?,Subalpine fir,SFVOL,1,0.0,0.0,0.0,0.0,...,426.9,427.9,428.8,429.6,430.4,431.2,431.4,431.4,431.4,431.4
4,31,*Y,?,Douglas-fir,DFVOL,1,0.0,0.0,0.0,0.0,...,118.2,118.5,118.7,118.9,119.1,119.3,119.5,119.5,119.4,119.4
5,23,*Y,?,Poplar,POVOL,1,0.0,0.0,0.3,3.0,...,232.6,233.6,234.6,235.5,236.4,236.9,237.0,237.0,237.0,237.0
6,1,*Y,?,Aspen,AAVOL,1,0.0,0.0,1.2,5.9,...,264.3,265.5,266.7,267.9,269.0,269.5,269.5,269.5,269.4,269.4
7,21,*Y,?,Spruce,SPVOL,1,0.0,0.0,0.0,0.0,...,469.7,467.4,465.2,463.1,461.1,459.2,458.5,458.3,458.3,458.3
8,2,*Y,?,Balsam fir,BFVOL,1,0.0,0.0,0.0,0.0,...,169.0,169.1,169.1,169.2,169.2,169.3,169.1,168.8,168.5,168.2
9,17,*Y,?,Spruce,SPVOL,1,0.0,0.0,0.0,0.0,...,153.7,153.6,153.4,153.3,153.2,153.2,153.1,153.1,153.0,153.0


Import CANFI tree species lookup table (associates tree species names with integer numerical values, which we use as theme data values in the ws3 model), and insert species code values into the yield curve dataframe.

In [ ]:
canf = pd.read_csv('data/canfi_species_modified.csv')
canf = canf[['name', 'canfi_species']].set_index('name')

Burn CANFI species codes into stand and yield data tables.

In [ ]:
stands['theme3'] = stands.apply(lambda row: canf.loc[row['theme3'], 'canfi_species'], axis=1) 
yldmerged['canfi_species'] = yldmerged.apply(lambda row: canf.loc[row['LDSPP'], 'canfi_species'], axis=1)

Add a new `curve_id` colume that has same data values as `AU` column.

In [ ]:
yldmerged['curve_id'] = yldmerged['AU'] 

Save reformatted data to CSV files. 

In [ ]:
yldmerged.to_csv('data/yldmerged.csv', header=True, index=False)
stands.to_csv('data/stands_table.csv', header=True, index=False)

Rename stuff to match variable names we expect further down.

In [ ]:
stands_table = stands
curve_points_table = yldmerged
curve_points_table.set_index('AU', inplace=True)

# Export Woodstock-formatted input files 

We can use the new ws3 model instance we just built to export ws3 input files in Woodstock file format. We do this for three reasons. 

The first reason is that it will be simpler and more compact in the actual DSS notebook to instantiate the `ForestModel` object from these Woodstock-formatted files (and also this will provide an opportunity to demonstrate the existance and usage of the Woodstock model import functions that are built into ws3). 

The second reason is that the process of exporting data from a live `ws3.forest.ForestModel` instance to Woodstock-formatted input data files provides some insight into the internal structure and workings of ws3 models (which can be a challenging thing to get started with, particularly if you do not have a lot of experience building and running forest estate models). 

The third reason is that Woodstock file format is designed to be "human readable" (sort of... nobody ever said it would be super easy or super fun). Picking through the exported Woodstock-formatted files might help some people better understand the structure and details of the model we have built. If you have no experience reading Woodstock-formatted model input data files, then this is going to be trickier (unless you pause here and go take an introductory Woodstock training course of sort). Many forest professionals already have familiarity with Woodstock software and its special file format (through having been exposed to this at some point in their career). 

Start by creating a new subdirectory to hold the new Woodstock-formatted data files.

In [ ]:
!mkdir data/woodstock_model_files

## LANDSCAPE section

The LANDSCAPE section defines stratification variables (themes) and stratification variable values (basecodes). 

In [ ]:
theme_cols=['theme0', # TSA 
            'theme1', # THLB
            'theme2', # AU
            'theme3', # leading species code
            'theme4'  # yield curve ID
           ]
basecodes = [list(map(lambda x: str(x), stands_table[tc].unique())) for tc in theme_cols]
basecodes[2] = list(set(basecodes[2] + list(stands_table['theme2'].astype(str))))
basecodes[3] = list(set(basecodes[3] + list(stands_table['theme3'].astype(str))))
basecodes[4] = list(set(basecodes[4] + list(stands_table['theme4'].astype(str))))

In [ ]:
with open('data/woodstock_model_files/tsa17.lan', 'w') as file:
    print('*THEME Timber Supply Area (TSA)', file=file)
    print('tsa17',file=file)
    print('*THEME Timber Harvesting Land Base (THLB)', file=file)
    for basecode in basecodes[1]: print(basecode, file=file)
    print('*THEME Analysis Unit (AU)', file=file)
    for basecode in basecodes[2]: print(basecode, file=file)
    print('*THEME Leading tree species (CANFI species code)', file=file)
    for basecode in basecodes[3]: print(basecode, file=file)
    print('*THEME Yield curve ID', file=file)
    for basecode in basecodes[4]: print(basecode, file=file)

## AREAS section

The AREAS section defines the initial forest inventory, in terms of how many hectares of which age class are present in which development type (where a development type is defined as a unique sequence of landscape theme variable values).

In [ ]:
gstands = stands_table.groupby(theme_cols+['age'])

In [ ]:
with open('data/woodstock_model_files/tsa17.are', 'w') as file:
    for name, group in gstands:
        dtk, age, area = tuple(map(lambda x: str(x), name[:-1])), int(name[-1]), group['area'].sum()
        print('*A', ' '.join(v for v in dtk), age, area, file=file)

## YIELDS section

The YIELDS section defines yield curves (in this example we only track merchantable log volume, but we can use yield curves to track all sorts of other stuff). 

In [ ]:
with open('data/woodstock_model_files/tsa17.yld', 'w') as file:
        tot=[]
        swd=[]
        hwd=[]
        for AU, au_row in curve_points_table.iterrows():
            yname = 's%04d' % int(au_row.canfi_species)    
            curve_id = au_row.curve_id
            mask = ('?', '?', str(AU), '?', str(curve_id))
            points = [(x*10, au_row['X%i' % (x*10)]) for x in range(36)]
            c = ws3.core.Curve(yname, points=points, type='a', is_volume=True, xmax=max_age, period_length=period_length)
            print('*Y', ' '.join(v for v in mask), file=file)
            print(yname, '1', ' '.join(str(int(c[x])) for x in range(0, 350, 10)), file=file)
            if yname not in tot:
                tot.append(yname)
            if int(au_row.canfi_species) > 1200:
                if yname not in hwd: hwd.append(yname)
            else:
                if yname not in swd: swd.append(yname)
        print('*YC ? ? ? ? ?', file=file)
        print('totvol _SUM(%s)' % ', '.join(map(str, tot)), file=file)
        print('swdvol _SUM(%s)' % ', '.join(map(str, swd)), file=file)
        print('hwdvol _SUM(%s)' % ', '.join(map(str, hwd)), file=file)

## ACTIONS section

The ACTIONS section defines actions that can be applied in the model (e.g., harvesting, planting, thinning, fertilization, etc). 

In [ ]:
with open('data/woodstock_model_files/tsa17.act', 'w') as file:
    print('ACTIONS', file=file)
    print('*ACTION harvest Y', file=file)
    print('*OPERABLE harvest', file=file)
    print('? 1 ? ? ? _AGE >= 90 AND _AGE <= 600', file=file)

## TRANSITIONS section

The TRANSITIONS section defines transitions (i.e., transition to a new development type and age class induced by applying a specific action to a specific combination of development type and age class). If there were no transitions in a forest estate model, it would simply be aging (i.e., growing) the forest forward from time step 1 through to time step N.

In [ ]:
with open('data/woodstock_model_files/tsa17.trn', 'w') as file:
    acode = 'harvest'
    print('*CASE', acode, file=file)
    record_au = set()
    for au_id, au_row in stands_table.iterrows():
        if au_row.theme2 in record_au: continue
        if not au_row.theme1: continue
        target_curve_id = au_row.theme4  
        smask = ' '.join(('?', '?' , str(target_curve_id), '?', '?'))
        tmask = ' '.join(('?', '?' , '?', '?', str(target_curve_id)))
        print('*SOURCE', smask, file=file)
        print('*TARGET', tmask, '100', file=file)
        record_au.add(au_row.theme2)